# Installs

In [ ]:
%pip install datasets

     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 122kB 14.9MB/s 
     |████████████████████████████████| 245kB 15.1MB/s 


In [ ]:
%pip install rouge_score

In [ ]:
%pip install sacrebleu

     |████████████████████████████████| 61kB 4.7MB/s 


In [ ]:
%pip install -U nltk

     |████████████████████████████████| 1.5MB 7.7MB/s 
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
%pip install git+https://github.com/google-research/bleurt.git

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-r7nl6cpf
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-r7nl6cpf
     |████████████████████████████████| 358kB 7.7MB/s 
     |████████████████████████████████| 1.2MB 13.2MB/s 
  Created wheel for BLEURT: filename=BLEURT-0.0.2-cp37-none-any.whl size=16454025 sha256=65359a8d478713eb1dbd65d16581c9502c9e63edb72f28ad18ee1df75fb99380
  Stored in directory: /tmp/pip-ephem-wheel-cache-84t2mcto/wheels/67/7a/5b/d9b15cc48bedca8bd1c444ac4d650467294980793d5dd7ba7c
Successfully built BLEURT


In [ ]:
%pip install bert_score

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 2.5MB 14.3MB/s 
     |████████████████████████████████| 901kB 49.1MB/s 
     |████████████████████████████████| 3.3MB 47.4MB/s 
  Found existing installation: huggingface-hub 0.0.13
    Uninstalling huggingface-hub-0.0.13:
      Successfully uninstalled huggingface-hub-0.0.13


In [ ]:
!git clone "https://github.com/neural-dialogue-metrics/Distinct-N.git"

Cloning into 'Distinct-N'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [ ]:
%pip install ./Distinct-N/

Processing ./Distinct-N
  Created wheel for Distinct-N: filename=Distinct_N-0.4.0-cp37-none-any.whl size=8788 sha256=c4a68cce34c36639de01f007947f8d70c5d6e5d8de28b355fd8814aa708743f8
  Stored in directory: /root/.cache/pip/wheels/49/61/89/0086f011267cc0bb1000649e733b256cfb87ec749c446ef0bf
Successfully built Distinct-N


In [ ]:
%pip install lexical-diversity

     |████████████████████████████████| 122kB 8.4MB/s 


In [ ]:
!git clone https://github.com/wl-research/nubia.git

Cloning into 'nubia'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 282 (delta 79), reused 150 (delta 69), pack-reused 116
Receiving objects: 100% (282/282), 1.71 MiB | 5.45 MiB/s, done.
Resolving deltas: 100% (123/123), done.


# Load dataset

In [ ]:
# Load dataset
from datasets import load_dataset
data = load_dataset("gem", "common_gen")

Dataset gem downloaded and prepared to /root/.cache/huggingface/datasets/gem/common_gen/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4. Subsequent calls will reuse this data.


## Explore dataset structure

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 67389
    })
    validation: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 993
    })
    test: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 1497
    })
    challenge_train_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_validation_sample: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
    challenge_test_scramble: Dataset({
        features: ['concept_set_id', 'concepts', 'gem_id', 'gem_parent_id', 'references', 'target'],
        num_rows: 500
    })
})

In [ ]:
test_instance = data['test'][0]
test_instance

In [ ]:
validation_instance = data['validation'][133]
validation_instance

In [ ]:
challenge_test_instance = data['challenge_test_scramble'][0]
challenge_test_instance

In [ ]:
challenge_validation_instance = data['challenge_validation_sample'][0]
challenge_validation_instance

In [ ]:
concept = challenge_validation_instance.get("target")
concept

In [ ]:
validation_targets = []
for i in range(501): 
  instance = data['validation'][i]
  target = instance.get("target")
  validation_targets.append(target)
validation_targets[500]

'The woman rides a bicycle down the street instead of walking.'

## Extract validation split

In [ ]:
validation_data = []
validation_data = data['validation']
print(validation_data['references'][0])
print(validation_data['target'][0])

['The player stood in the field looking at the batter.', 'The coach stands along the field, looking at the goalkeeper.', 'I stood and looked across the field, peacefully.', 'Someone stands, looking around the empty field.']
The player stood in the field looking at the batter.


## Clean generated text

In [ ]:
# Clean validation greedy sampling generation
data_file = "sample_data/news_greedy_validation.txt"
greedy_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    greedy_generated_data.append(complete_sentence)

greedy_generated_data[0]

'it was just one guy off the field and everything , but you have to look after me and i and my sisters stand out.'

In [ ]:
# Clean validation top-k generation
data_file = "sample_data/news_sampling_validation.txt"
sampling_generated_data = []

with open(data_file, 'r') as f:
  for i, line in enumerate(f):
    line = line.strip()
    example = line.split()
    new_example = []
    for word in example:
      if word.startswith("[unused") or word.startswith("[UNK]") or word.startswith(".") or word.startswith("\\") or word.startswith("##"):
        del word
      else:
        new_example.append(word)

    complete_sentence = " ".join(new_example) + "."
    sampling_generated_data.append(complete_sentence)

sampling_generated_data[0]

'there was only one person on the field that this is what you look like , and i believe it to stand out.'

# Compute metrics


## Lexical (ROUGE 1/2/L, BLEU, Meteor), 

In [ ]:
from datasets import list_metrics, load_metric
import numpy as np

metrics_list = list_metrics()
print(metrics_list)

['accuracy', 'bertscore', 'bleu', 'bleurt', 'cer', 'comet', 'coval', 'cuad', 'f1', 'gleu', 'glue', 'indic_glue', 'matthews_correlation', 'meteor', 'pearsonr', 'precision', 'recall', 'rouge', 'sacrebleu', 'sari', 'seqeval', 'spearmanr', 'squad', 'squad_v2', 'super_glue', 'wer', 'xnli']


In [ ]:
rouge_scorer = load_metric("rouge")
sacrebleu_scorer = load_metric("sacrebleu")
meteor_scorer = load_metric("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
# ROUGE target
greedy_rouge_results = rouge_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Greedy rouge1", round(greedy_rouge_results['rouge1'].mid.fmeasure, 4))
print("Greedy rouge2", round(greedy_rouge_results['rouge2'].mid.fmeasure, 4))
print("Greedy rougeL", round(greedy_rouge_results['rougeL'].mid.fmeasure, 4))


sampling_rouge_results = rouge_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    rouge_types=["rouge1", "rouge2", "rougeL"],
    use_agregator=True, use_stemmer=False,
)

print("Sampling rouge1", round(sampling_rouge_results['rouge1'].mid.fmeasure, 4))
print("Sampling rouge2", round(sampling_rouge_results['rouge2'].mid.fmeasure, 4))
print("Sampling rougeL", round(sampling_rouge_results['rougeL'].mid.fmeasure, 4))

Greedy rouge1 0.2913
Greedy rouge2 0.0576
Greedy rougeL 0.2028
Sampling rouge1 0.2891
Sampling rouge2 0.0573
Sampling rougeL 0.2006


In [ ]:
greedy_rouge_results

{'rouge1': AggregateScore(low=Score(precision=0.2130725178099787, recall=0.4534288410372081, fmeasure=0.28547640481744463), mid=Score(precision=0.21791028535748402, recall=0.46447063250807924, fmeasure=0.2913297695542487), high=Score(precision=0.2223647148376101, recall=0.4750496628544785, fmeasure=0.297000105850806)),
 'rouge2': AggregateScore(low=Score(precision=0.03943150421772314, recall=0.08899798077595211, fmeasure=0.053747353738254434), mid=Score(precision=0.04231855442021719, recall=0.09603667811935032, fmeasure=0.05759837901811514), high=Score(precision=0.04506797738823927, recall=0.10284671615618514, fmeasure=0.06134176593837011)),
 'rougeL': AggregateScore(low=Score(precision=0.1481651654878542, recall=0.3175676158455767, fmeasure=0.1987912293778941), mid=Score(precision=0.15137598092319923, recall=0.3244399387676016, fmeasure=0.20281953582574858), high=Score(precision=0.15469663076400167, recall=0.3327765454536725, fmeasure=0.20711086233429624))}

In [ ]:
# BLEU target

greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[[validation_data['target'][i]]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 3.702294443546175
Sampling BLEU mean 3.616723243730503


In [ ]:
# BLEU references
greedy_bleu_results = []
for i, sentence in enumerate(zip(greedy_generated_data, validation_data)):
  greedy_bleu_result = sacrebleu_scorer.compute(
      predictions=[greedy_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  greedy_bleu_results.append(greedy_bleu_result["score"])

sampling_bleu_results = []
for i, sentence in enumerate(zip(sampling_generated_data, validation_data)):
  sampling_bleu_result = sacrebleu_scorer.compute(
      predictions=[sampling_generated_data[i]],
      references=[validation_data['references'][i]]
  )
  sampling_bleu_results.append(sampling_bleu_result["score"])

greedy_bleu_mean = np.mean(greedy_bleu_results)
sampling_bleu_mean = np.mean(sampling_bleu_results)

print("Greedy BLEU mean", greedy_bleu_mean)
print("Sampling BLEU mean", sampling_bleu_mean)


Greedy BLEU mean 5.58000498036059
Sampling BLEU mean 5.4054633831328


In [ ]:
### NIST score
from nltk.translate import nist_score

reference = 'un test simple'.split()
reference1 = 'un test super simple'.split()
candidate = 'un test pas trop simple'.split()

print('Sentence 1-gram: %f' % nist_score.sentence_nist([reference, reference1], candidate, 1))
print('Sentence 2-gram: %f' % nist_score.sentence_nist([reference], candidate, 2))
print('Sentence 3-gram: %f' % nist_score.sentence_nist([reference], candidate, 3))
print('Sentence 4-gram: %f' % nist_score.sentence_nist([reference], candidate, 4))
print('Sentence 5-gram: %f' % nist_score.sentence_nist([reference], candidate, 5))

print('Corpus 1-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 1))
print('Corpus 2-gram: %f' % nist_score.corpus_nist([[reference, reference1]], [candidate], 2))
print('Corpus 3-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 3))
print('Corpus 4-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 4))
print('Corpus 5-gram: %f' % nist_score.corpus_nist([[reference]], [candidate], 5))

Sentence 1-gram: 1.084413
Sentence 2-gram: 0.950978
Sentence 3-gram: 0.950978
Sentence 4-gram: 0.950978
Sentence 5-gram: 0.950978
Corpus 1-gram: 1.084413
Corpus 2-gram: 1.084413
Corpus 3-gram: 0.950978
Corpus 4-gram: 0.950978
Corpus 5-gram: 0.950978


In [ ]:
# NIST
from nltk.translate import nist_score

greedy_nist_results = []
for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = greedy_generated_data[i].split()
  
  if len(candidate) > 4:
    greedy_nist_result = nist_score.sentence_nist(reference, candidate)
    greedy_nist_results.append(greedy_nist_result)

sampling_nist_results = []
for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  reference = validation_data['target'][i].split()
  candidate = sampling_generated_data[i].split()
  
  if len(candidate) > 4:
    sampling_nist_result = nist_score.sentence_nist(reference, candidate)
    sampling_nist_results.append(sampling_nist_result)

    
greedy_nist_mean = np.mean(greedy_nist_results)
sampling_nist_mean = np.mean(sampling_nist_results)

print("Greedy NIST mean", greedy_nist_mean)
print("Sampling NIST mean", sampling_nist_mean)

Greedy NIST mean 0.15877491123833568
Sampling NIST mean 0.15116051923567042


In [ ]:
# METEOR
greedy_meteor_results = meteor_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)

sampling_meteor_results = meteor_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

print("Greedy meteor", round(greedy_meteor_results['meteor'], 4))
print("Sampling meteor", round(sampling_meteor_results['meteor'], 4))

Greedy meteor 0.2592
Sampling meteor 0.2584


 ## Semantic (BERTscore, BLEURT, NUBIA)

In [ ]:
import numpy as np

bleurt_scorer = load_metric('bleurt', 'bleurt-large-512')
bertscore_scorer = load_metric("bertscore")


INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/bleurt-large-512/downloads/extracted/299e33e80b83c78cc60e485384c7804f6ec1fb36c2013c5078257c17a82719ca/bleurt-large-512.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


In [ ]:
# BLEURT

greedy_bleurt_results = bleurt_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target']
)
sampling_bleurt_results = bleurt_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target']
)

In [ ]:
# When sequence length is 512
greedy_bleurt_results

{'scores': [-1.1124281883239746,
  -1.1951218843460083,
  -0.9247578978538513,
  -0.6957359910011292,
  -1.1802446842193604,
  -0.8080786466598511,
  -0.6463359594345093,
  -0.5422631502151489,
  -1.1425994634628296,
  -0.6893836259841919,
  -1.0147522687911987,
  -1.1905848979949951,
  -0.8980950117111206,
  -0.6821208596229553,
  -0.27533993124961853,
  -0.7760045528411865,
  -0.7218672633171082,
  -1.2088316679000854,
  -0.9786050319671631,
  -1.1314454078674316,
  -0.8383071422576904,
  -1.1250369548797607,
  -1.425742506980896,
  -0.9938899874687195,
  -0.2961623966693878,
  -1.271300196647644,
  -0.885051429271698,
  -0.8665528297424316,
  -1.0129692554473877,
  -1.0768252611160278,
  -1.2124565839767456,
  -1.0993355512619019,
  -0.6890954375267029,
  -0.6489330530166626,
  -0.6884561777114868,
  -0.8666375279426575,
  -0.7983871698379517,
  -0.9549688696861267,
  -1.0777385234832764,
  -0.7863006591796875,
  -0.8050259351730347,
  -0.6008592844009399,
  -0.7132245302200317,
  -

In [ ]:
# When seq len = 512
sampling_bleurt_results

{'scores': [-0.652430534362793,
  -0.7639352083206177,
  -1.208678126335144,
  -0.5746899843215942,
  -1.2782204151153564,
  -1.0859248638153076,
  -1.0966882705688477,
  -0.7134405374526978,
  -0.9913756847381592,
  -0.7616559267044067,
  -0.696890115737915,
  -1.0914998054504395,
  -0.8424816727638245,
  -0.8117647767066956,
  -0.9089556932449341,
  -0.6035383343696594,
  -0.576862096786499,
  -1.1088337898254395,
  -0.9710656404495239,
  -1.2846335172653198,
  -0.8199719190597534,
  -1.2444623708724976,
  -1.2639775276184082,
  -1.0738738775253296,
  -0.4125197231769562,
  -0.9520834684371948,
  -0.592653751373291,
  -0.8441427946090698,
  -0.9042997360229492,
  -0.6057102680206299,
  -1.2608832120895386,
  -0.9387394785881042,
  -1.1566907167434692,
  -0.368415504693985,
  -0.9229270815849304,
  -0.7980943918228149,
  -0.9589431285858154,
  -0.7271215319633484,
  -1.3272508382797241,
  -0.2549841105937958,
  -0.6766852140426636,
  -0.5567073822021484,
  -0.8963181972503662,
  -0.87

In [ ]:
greedy_bleurt_mean = np.mean(greedy_bleurt_results['scores'])
sampling_bleurt_mean = np.mean(sampling_bleurt_results['scores'])

print("Greedy bleurt mean", greedy_bleurt_mean)
print("Sampling bleurt mean", sampling_bleurt_mean)

Greedy bleurt mean -0.8283402523155299
Sampling bleurt mean -0.8671864661034515


In [ ]:
print("Min greedy value: ", np.min(greedy_bleurt_results.get('scores')))
print("Max greedy value: ", np.max(greedy_bleurt_results.get('scores')))

Min greedy value:  -1.501287579536438
Max greedy value:  0.11801104247570038


In [ ]:
print("Min sampling value: ", np.min(sampling_bleurt_results.get('scores')))
print("Max sampling value: ", np.max(sampling_bleurt_results.get('scores')))

Min sampling value:  -1.516877293586731
Max sampling value:  0.1376999467611313


In [ ]:
# BERTscore

greedy_bertscore_results = bertscore_scorer.compute(
    predictions=greedy_generated_data,
    references=validation_data['target'],
    lang="en"
)

sampling_bertscore_results = bertscore_scorer.compute(
    predictions=sampling_generated_data,
    references=validation_data['target'],
    lang="en"
)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
greedy_bertscore_results

{'f1': [0.8516578078269958,
  0.8621100783348083,
  0.87276291847229,
  0.8985835909843445,
  0.8744574189186096,
  0.8806823492050171,
  0.864368736743927,
  0.8819712996482849,
  0.8631078004837036,
  0.8642768859863281,
  0.8593399524688721,
  0.8639323711395264,
  0.8519476652145386,
  0.8757831454277039,
  0.8772350549697876,
  0.873245358467102,
  0.880850613117218,
  0.8231006860733032,
  0.8985863327980042,
  0.835200309753418,
  0.8827697038650513,
  0.8590741157531738,
  0.8561353087425232,
  0.8541973829269409,
  0.8851224184036255,
  0.8844680786132812,
  0.8800967335700989,
  0.8901497721672058,
  0.862712025642395,
  0.8800878524780273,
  0.8585731387138367,
  0.8643640875816345,
  0.8772820234298706,
  0.8589531183242798,
  0.855962336063385,
  0.8742722272872925,
  0.8848018050193787,
  0.8963018655776978,
  0.8513659238815308,
  0.8941022753715515,
  0.8920078873634338,
  0.8889518976211548,
  0.891817569732666,
  0.8609216809272766,
  0.8770961165428162,
  0.871571719

In [ ]:
greedy_bertscore_f1_mean = np.mean(greedy_bertscore_results['f1'])
greedy_bertscore_precision_mean = np.mean(greedy_bertscore_results['precision'])
greedy_bertscore_recall_mean = np.mean(greedy_bertscore_results['recall'])

sampling_bertscore_f1_mean = np.mean(sampling_bertscore_results['f1'])
sampling_bertscore_precision_mean = np.mean(sampling_bertscore_results['precision'])
sampling_bertscore_recall_mean = np.mean(sampling_bertscore_results['recall'])

print("Greedy F1 mean", round(greedy_bertscore_f1_mean, 4))
print("Greedy precision mean", round(greedy_bertscore_precision_mean, 4))
print("Greedy recall mean", round(greedy_bertscore_recall_mean, 4))

print("Sampling F1 mean", round(sampling_bertscore_f1_mean, 4))
print("Sampling precision mean", round(sampling_bertscore_precision_mean, 4))
print("Sampling recall mean", round(sampling_bertscore_recall_mean, 4))


Greedy F1 mean 0.8689
Greedy precision mean 0.8504
Greedy recall mean 0.8883
Sampling F1 mean 0.868
Sampling precision mean 0.8489
Sampling recall mean 0.8881


In [ ]:
import os
os.chdir('nubia')

In [ ]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.7MB 7.6MB/s 
     |████████████████████████████████| 184kB 43.6MB/s 
     |████████████████████████████████| 2.4MB 34.3MB/s 
     |████████████████████████████████| 153kB 45.1MB/s 
     |████████████████████████████████| 133kB 45.4MB/s 
     |████████████████████████████████| 215kB 50.0MB/s 
     |████████████████████████████████| 1.9MB 38.0MB/s 
     |████████████████████████████████| 112kB 54.1MB/s 
     |████████████████████████████████| 81kB 10.8MB/s 
     |████████████████████████████████| 7.7MB 12.3MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
     |████████████████████████████████| 3.2MB 39.2MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 962kB 38.8MB/s 
     |████████████████████████████████| 645kB 37.8MB/s 
  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9675 sha256=890eeebea9850d9c34e88a368092bc3f217fe52152d17c0c0c5119f46d6dff53
  Stored in director

In [ ]:
from nubia_score import Nubia
n = Nubia()

1042301B [00:00, 5449937.42B/s]
456318B [00:00, 4174707.53B/s]
100%|██████████| 548118077/548118077 [00:14<00:00, 37981008.43B/s]


In [ ]:
greedy_nubia_contradiction = []
greedy_nubia_grammar_hyp = []
greedy_nubia_grammar_ref = []
greedy_nubia_logical_agreement = []
greedy_nubia_semantic_relation = []
greedy_nubia_score = []

for i, sentence in enumerate(zip(validation_data, greedy_generated_data)):
  greedy_nubia_result = n.score(
      greedy_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  greedy_nubia_contradiction.append(greedy_nubia_result.get('features').get('contradiction'))
  greedy_nubia_grammar_hyp.append(greedy_nubia_result.get('features').get('grammar_hyp'))
  greedy_nubia_grammar_ref.append(greedy_nubia_result.get('features').get('grammar_ref'))
  greedy_nubia_logical_agreement.append(greedy_nubia_result.get('features').get('logical_agreement'))
  greedy_nubia_semantic_relation.append(greedy_nubia_result.get('features').get('semantic_relation'))

  greedy_nubia_score.append(greedy_nubia_result.get('nubia_score'))

Streaming output truncated to the last 5000 lines.

NUBIA score: 0.08100644478994201/1.0
Semantic relation: 1.3958995342254639/5.0
Percent chance of contradiction: 74.55829977989197%
Percent chance of irrelevancy or new information: 10.407861322164536%
Percent chance of logical agreement: 15.033838152885437%

NUBIA score: 0.0817228894573021/1.0
Semantic relation: 1.523128628730774/5.0
Percent chance of contradiction: 1.5438986010849476%
Percent chance of irrelevancy or new information: 96.1188018321991%
Percent chance of logical agreement: 2.3372961208224297%

NUBIA score: 0.12853039425678747/1.0
Semantic relation: 2.118032693862915/5.0
Percent chance of contradiction: 10.954724252223969%
Percent chance of irrelevancy or new information: 85.30048727989197%
Percent chance of logical agreement: 3.7447862327098846%

NUBIA score: 0.10838493711259711/1.0
Semantic relation: 1.689657211303711/5.0
Percent chance of contradiction: 93.92900466918945%
Percent chance of irrelevancy or new informat

In [ ]:
import numpy as np

greedy_nubia_contradiction
greedy_nubia_grammar_hyp
greedy_nubia_grammar_ref
greedy_nubia_logical_agreement
greedy_nubia_semantic_relation
greedy_nubia_score

greedy_nubia_contradiction_mean = np.mean(greedy_nubia_contradiction)
greedy_nubia_grammar_hyp_mean = np.mean(greedy_nubia_grammar_hyp)
greedy_nubia_grammar_ref_mean = np.mean(greedy_nubia_grammar_ref)
greedy_nubia_logical_agreement_mean = np.mean(greedy_nubia_logical_agreement)
greedy_nubia_semantic_relation_mean = np.mean(greedy_nubia_semantic_relation)
greedy_nubia_score_mean = np.mean(greedy_nubia_score)

print("Greedy nubia_contradiction mean", round(greedy_nubia_contradiction_mean, 4))
print("Greedy nubia_grammar_hyp mean", round(greedy_nubia_grammar_hyp_mean, 4))
print("Greedy nubia_grammar_ref mean", round(greedy_nubia_grammar_ref_mean, 4))
print("Greedy nubia_logical_agreement mean", round(greedy_nubia_logical_agreement_mean, 4))
print("Greedy nubia_semantic_relation mean", round(greedy_nubia_semantic_relation_mean, 4))
print("Greedy nubia_score mean", round(greedy_nubia_score_mean, 4))

Greedy nubia_contradiction mean 26.4468
Greedy nubia_grammar_hyp mean 4.5859
Greedy nubia_grammar_ref mean 4.6839
Greedy nubia_logical_agreement mean 19.6621
Greedy nubia_semantic_relation mean 1.9796
Greedy nubia_score mean 0.1364


In [ ]:
sampling_nubia_contradiction = []
sampling_nubia_grammar_hyp = []
sampling_nubia_grammar_ref = []
sampling_nubia_logical_agreement = []
sampling_nubia_semantic_relation = []
sampling_nubia_score = []

for i, sentence in enumerate(zip(validation_data, sampling_generated_data)):
  sampling_nubia_result = n.score(
      sampling_generated_data[i],
      validation_data['target'][i],
      verbose=True, 
      get_features=True
  )

  sampling_nubia_contradiction.append(sampling_nubia_result.get('features').get('contradiction'))
  sampling_nubia_grammar_hyp.append(sampling_nubia_result.get('features').get('grammar_hyp'))
  sampling_nubia_grammar_ref.append(sampling_nubia_result.get('features').get('grammar_ref'))
  sampling_nubia_logical_agreement.append(sampling_nubia_result.get('features').get('logical_agreement'))
  sampling_nubia_semantic_relation.append(sampling_nubia_result.get('features').get('semantic_relation'))

  sampling_nubia_score.append(sampling_nubia_result.get('nubia_score'))

Streaming output truncated to the last 5000 lines.

NUBIA score: 0.080061244096573/1.0
Semantic relation: 2.331080675125122/5.0
Percent chance of contradiction: 1.617526262998581%
Percent chance of irrelevancy or new information: 45.84196209907532%
Percent chance of logical agreement: 52.54051685333252%

NUBIA score: 0.18627296755260092/1.0
Semantic relation: 1.4789479970932007/5.0
Percent chance of contradiction: 41.77188277244568%
Percent chance of irrelevancy or new information: 58.02096724510193%
Percent chance of logical agreement: 0.20714306738227606%

NUBIA score: 0.10431442545784508/1.0
Semantic relation: 2.0833799839019775/5.0
Percent chance of contradiction: 2.007221058011055%
Percent chance of irrelevancy or new information: 97.28720784187317%
Percent chance of logical agreement: 0.7055672816932201%

NUBIA score: 0.10461071032689284/1.0
Semantic relation: 1.8977272510528564/5.0
Percent chance of contradiction: 98.23153018951416%
Percent chance of irrelevancy or new informati

In [ ]:
sampling_nubia_contradiction
sampling_nubia_grammar_hyp
sampling_nubia_grammar_ref
sampling_nubia_logical_agreement
sampling_nubia_semantic_relation
sampling_nubia_score

sampling_nubia_contradiction_mean = np.mean(sampling_nubia_contradiction)
sampling_nubia_grammar_hyp_mean = np.mean(sampling_nubia_grammar_hyp)
sampling_nubia_grammar_ref_mean = np.mean(sampling_nubia_grammar_ref)
sampling_nubia_logical_agreement_mean = np.mean(sampling_nubia_logical_agreement)
sampling_nubia_semantic_relation_mean = np.mean(sampling_nubia_semantic_relation)
sampling_nubia_score_mean = np.mean(sampling_nubia_score)

print("Sampling nubia_contradiction mean", round(sampling_nubia_contradiction_mean, 4))
print("Sampling nubia_grammar_hyp mean", round(sampling_nubia_grammar_hyp_mean, 4))
print("Sampling nubia_grammar_ref mean", round(sampling_nubia_grammar_ref_mean, 4))
print("Sampling nubia_logical_agreement mean", round(sampling_nubia_logical_agreement_mean, 4))
print("Sampling nubia_semantic_relation mean", round(sampling_nubia_semantic_relation_mean, 4))
print("Sampling nubia_score mean", round(sampling_nubia_score_mean, 4))

Sampling nubia_contradiction mean 25.186
Sampling nubia_grammar_hyp mean 4.5859
Sampling nubia_grammar_ref mean 4.8389
Sampling nubia_logical_agreement mean 20.2178
Sampling nubia_semantic_relation mean 1.928
Sampling nubia_score mean 0.1372


## Diversity (MSTTR, Distinct 1/2/3, Unique 1/2/3, Entropy 1/2/3)


In [ ]:
# Mean segmental Type-Token Ratio (MSTTR)
from lexical_diversity import lex_div as ld

greedy_msttr_results = []
for i, sentence in enumerate(greedy_generated_data):
  greedy_flt = ld.flemmatize(greedy_generated_data[i])
  greedy_msttr_result = ld.msttr(greedy_flt)
  greedy_msttr_results.append(greedy_msttr_result)

sampling_msttr_results = []
for i, sentence in enumerate(sampling_generated_data):
  sampling_flt = ld.flemmatize(sampling_generated_data[i])
  sampling_msttr_result = ld.msttr(sampling_flt)
  sampling_msttr_results.append(sampling_msttr_result)


greedy_msttr_mean = np.mean(greedy_msttr_results)
sampling_msttr_mean = np.mean(sampling_msttr_results)

print("Greedy MSTTR mean", greedy_msttr_mean)
print("Sampling MSTTR ", sampling_msttr_mean)

Greedy MSTTR mean 0.8643861677691339
Sampling MSTTR  0.8721394030008475


In [ ]:
# Distinct-n
from distinct_n import distinct_n_corpus_level

greedy_distinct_1_results = distinct_n_corpus_level(greedy_generated_data, 1)
sampling_distinct_1_results = distinct_n_corpus_level(sampling_generated_data, 1)

greedy_distinct_2_results = distinct_n_corpus_level(greedy_generated_data, 2)
sampling_distinct_2_results = distinct_n_corpus_level(sampling_generated_data, 2)

greedy_distinct_3_results = distinct_n_corpus_level(greedy_generated_data, 3)
#sampling_distinct_3_results = distinct_n_corpus_level(sampling_generated_data, 3)

print("Greedy distict 1-gram", greedy_distinct_1_results)
print("Greedy distict 2-gram", greedy_distinct_2_results)
print("Greedy distict 3-gram", greedy_distinct_3_results)

print("Sampling distict 1-gram", sampling_distinct_1_results)
print("Sampling distict 2-gram", sampling_distinct_2_results)
#print("Sampling distict 3-gram", sampling_distinct_3_results)

Greedy distict 1-gram 0.2049885918944677
Greedy distict 2-gram 0.6332431180691296
Greedy distict 3-gram 0.8405759149679094
Sampling distict 1-gram 0.19896850377531763
Sampling distict 2-gram 0.6351406431759313


In [ ]:
# Unique-n

import nltk

greedy_document = []
for sentence in greedy_generated_data:
  sentence = sentence.split()
  for word in sentence:
    greedy_document.append(word)

sampling_document = []
for sentence in sampling_generated_data:
  sentence = sentence.split()
  for word in sentence:
    sampling_document.append(word)

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

def unique_finder(finder):
  counter = 0
  for occurence in range(len(finder)):
    item = finder.popitem()
    if item[1] == 1:
      counter += 1
  return counter

greedy_uni_count = unique_finder(greedy_unigram_finder)
greedy_bi_count = unique_finder(greedy_bigram_finder)
greedy_tri_count = unique_finder(greedy_trigram_finder)

sampling_uni_count = unique_finder(sampling_unigram_finder)
sampling_bi_count = unique_finder(sampling_bigram_finder)
sampling_tri_count = unique_finder(sampling_trigram_finder)

print("Greedy unique unigrams", greedy_uni_count)
print("Greedy unique bigrams", greedy_bi_count)
print("Greedy unique trigrams", greedy_tri_count)

print("Sampling unique unigrams", sampling_uni_count)
print("Sampling unique bigrams", sampling_bi_count)
print("Sampling unique trigrams", sampling_tri_count)

Greedy unique unigrams 1089
Greedy unique bigrams 9954
Greedy unique trigrams 19776
Sampling unique unigrams 1135
Sampling unique bigrams 10934
Sampling unique trigrams 21241


In [47]:
# Entropy-n
from scipy import stats

greedy_unigram_finder = nltk.FreqDist(greedy_document)
greedy_bigram_finder = nltk.FreqDist(nltk.bigrams(greedy_document))
greedy_trigram_finder = nltk.FreqDist(nltk.trigrams(greedy_document))

sampling_unigram_finder = nltk.FreqDist(sampling_document)
sampling_bigram_finder = nltk.FreqDist(nltk.bigrams(sampling_document))
sampling_trigram_finder = nltk.FreqDist(nltk.trigrams(sampling_document))

greedy_unigram_probabilities = [v for k,v in greedy_unigram_finder.items()]
greedy_bigram_probabilities = [v for k,v in greedy_bigram_finder.items()]
greedy_trigram_probabilities = [v for k,v in greedy_trigram_finder.items()]

sampling_unigram_probabilities = [v for k,v in sampling_unigram_finder.items()]
sampling_bigram_probabilities = [v for k,v in sampling_bigram_finder.items()]
sampling_trigram_probabilities = [v for k,v in sampling_trigram_finder.items()]

greedy_entropy_1 = stats.entropy(greedy_unigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_bigram_probabilities)
greedy_entropy_2 = stats.entropy(greedy_trigram_probabilities)

sampling_entropy_1 = stats.entropy(sampling_unigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_bigram_probabilities)
sampling_entropy_2 = stats.entropy(sampling_trigram_probabilities)

print("Greedy entropy-1:", greedy_entropy_1)
print("Greedy entropy-2:", greedy_entropy_2)
#print("Greedy entropy-3:", greedy_entropy_3)

print("Sampling entropy-1:", sampling_entropy_1)
print("Sampling entropy-2:", sampling_entropy_2)
#print("Sampling entropy-3:", sampling_entropy_3)

Greedy entropy-1: 5.808616314634877
Greedy entropy-2: 9.855827387399478
Sampling entropy-1: 5.893868240254596
Sampling entropy-2: 9.944780363017966
